# SETUP CELLS

In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
import requests
import json

# === Setup Paths ===
sys.path.insert(0, os.path.abspath("./genie-worksheets/src"))
root_dir = os.getcwd()

# === Load Environment ===
env_file = os.path.join(root_dir, ".env")
load_dotenv(env_file)

# === Verify API Key ===
AGENT_KEY = os.getenv("LLM_API_KEY")
BASE_URL = os.getenv("LLM_API_BASE_URL")
DATABASE_KEY = os.getenv("EXA_API_KEY")

# print(AGENT_KEY)


if not AGENT_KEY or AGENT_KEY == "your_api_key_here":
    raise ValueError("❌ Please set your LLM_API_KEY in .env file")

print("✓ Environment loaded")
print(f"✓ API Base URL: {BASE_URL}")


# === Import only what you need from Genie ===
from worksheets.specification.from_spreadsheet import gsheet_to_classes
print("✓ Genie Worksheets imported")

✓ Environment loaded
✓ API Base URL: https://api.openai.com/v1
✓ Genie Worksheets imported


In [2]:
# Ensure Environment is actually working
import requests
import json
from dotenv import load_dotenv

response = requests.post(
    f"{BASE_URL}/chat/completions",
    headers={
        "Authorization": f"Bearer {AGENT_KEY}",
        "Content-Type": "application/json"
    },
    json={
        "model": "gpt-4o-mini",
        "messages": [
            {"role": "user", "content": "Say hi in one word"}
        ],
        "max_tokens": 5
    }
)

if response.status_code == 200:
    result = response.json()
    print("Success!")
    print(f"Response: {result['choices'][0]['message']['content']}")
    print(f"Tokens used: {result.get('usage', {}).get('total_tokens', 'N/A')}")
else:
    print(f"Error {response.status_code}: {response.text}")

Success!
Response: Hello!
Tokens used: 14


In [3]:
current_dir = os.getcwd()
root_dir = os.path.join(current_dir, "genie-worksheets")

# Credentials for reading the spreadsheet
!curl -L -o creds.zip "https://drive.google.com/uc?export=download&id=11QvSs2JZ5qpPrCvbX66Dg8pxfM_B8GaH"
!unzip creds.zip -d genie-worksheets/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3009  100  3009    0     0   2836      0  0:00:01  0:00:01 --:--:--     0
Archive:  creds.zip
  inflating: genie-worksheets//worksheets/token.json  
  inflating: genie-worksheets//worksheets/credentials.json  
  inflating: genie-worksheets//worksheets/service_account.json  


In [4]:
!mv "genie-worksheets/worksheets/credentials.json" "genie-worksheets/src/worksheets/config/credentials.json"
!mv "genie-worksheets/worksheets/service_account.json" "genie-worksheets/src/worksheets/config/service_account.json"
!mv "genie-worksheets/worksheets/token.json" "genie-worksheets/src/worksheets/config/token.json"

creds1 = os.path.join(root_dir, "src", "worksheets", "config", "credentials.json")
creds2 = os.path.join(root_dir, "src", "worksheets", "config", "service_account.json")
creds3 = os.path.join(root_dir, "src", "worksheets", "config", "token.json")

for cred in [creds1, creds2, creds3]:
  assert os.path.exists(cred), f"Cannot find the credential file: {cred}"

In [5]:
# Load collectibles from JSON file
def load_collectibles_from_json(json_path: str) -> list:
    """Load and convert collectibles from JSON file to the expected format."""
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    collectibles = []
    for item in data.get('collectibles', []):
        # Combine all instructions from different sources into one comprehensive instruction
        # We'll use the first source as the primary instruction
        primary_instruction = item['info'][0]['instruction'] if item['info'] else ""
        
        collectible = {
            "collectible_id": item['collectible_id'],
            "collectible_type": item['collectible_type'],
            "collectible_number": item['collectible_number'],
            "hidden": item['hidden'],
            "section": item['section'],
            "full_instruction": primary_instruction,
            "source": item['info'][0]['source'] if item['info'] else "Unknown"
        }
        collectibles.append(collectible)
    
    return collectibles

# Load collectibles from the JSON file
json_file_path = os.path.join(current_dir, "final_chapter1_info.json")
CELESTE_COLLECTIBLES = load_collectibles_from_json(json_file_path)

print(f"✓ Loaded {len(CELESTE_COLLECTIBLES)} collectibles from Chapter 1")


✓ Loaded 22 collectibles from Chapter 1


In [6]:
from worksheets.agent.config import agent_api
import requests
import json

# Generate three tiers of hints using provided LLM model
def generate_tiered_hints(full_instruction: str, collectible_type: str) -> list[str]:
    """
    Use LLM to break down a full instruction into 3 progressively revealing hint tiers.
    
    Args:
        full_instruction: The complete instruction for getting the collectible
        collectible_type: Type of collectible (e.g., "strawberry")
    
    Returns:
        List of 3 hint strings, from vague to specific
    """
    prompt = f"""You are helping a player find a {collectible_type} collectible in the game Celeste. 

Given the following complete instruction, break it down into 3 progressively revealing hint tiers:

**Tier 1 (Vague)**: Give only a general hint about the location or what to look for. Don't give away specific actions.
**Tier 2 (Medium)**: Provide more specific guidance about the general approach and key mechanics needed, but don't give step-by-step instructions.
**Tier 3 (Detailed)**: Provide the complete, step-by-step instruction.

Full Instruction:
{full_instruction}

Respond in JSON format with this exact structure:
{{
  "tier_1": "your vague hint here",
  "tier_2": "your medium hint here", 
  "tier_3": "your detailed hint here"
}}

Only return the JSON, nothing else."""

    try:
        response = requests.post(
            f"{BASE_URL}/chat/completions",
            headers={
                "Authorization": f"Bearer {AGENT_KEY}",
                "Content-Type": "application/json"
            },
            json={
                "model": "gpt-4o-mini",
                "messages": [
                    {"role": "user", "content": prompt}
                ],
                "temperature": 0.7,
                "max_tokens": 500
            },
            timeout=30
        )
        
        if response.status_code == 200:
            result = response.json()
            content = result['choices'][0]['message']['content'].strip()
            
            # Parse JSON response
            # Remove markdown code blocks if present
            if content.startswith("```"):
                content = content.split("```")[1]
                if content.startswith("json"):
                    content = content[4:].strip()
            
            hints_data = json.loads(content)
            return [
                hints_data.get("tier_1", ""),
                hints_data.get("tier_2", ""),
                hints_data.get("tier_3", full_instruction)  # Fallback to full instruction
            ]
        else:
            # Fallback: return the full instruction for all tiers
            return [
                "Look for hidden or unusual areas in this section.",
                "Pay attention to breakable walls and platforms.",
                full_instruction
            ]
    except Exception as e:
        print(f"Error generating tiered hints: {e}")
        # Fallback
        return [
            "Explore carefully and look for hidden paths.",
            "Use your dash ability to access hard-to-reach areas.",
            full_instruction
        ]

In [70]:
# Cache for generated hints to avoid repeated API calls
_hint_cache = {}

@agent_api("get_next_strawberry_id", "Get the ID of the next strawberry based on count")
def get_next_strawberry_id(current_count: str):
    # If I have 4, I want the 5th one
    next_num = int(current_count) + 1
    return f"strawberry_{next_num}"

# Function for genie worksheet to get a tiered hint for Celeste collectibles
@agent_api("get_hint", "Get a hint for a Celeste collectible")
def get_hint(collectible_id: str, hint_tier: int):
    """
    Get a hint for a specific collectible.

    Args:
        collectible_id: The collectible ID (e.g., "strawberry_11")
        hint_tier: Which hint tier to return (1, 2, or 3)
    
    Returns:
        A formatted hint message string
    """
    if collectible_id.isdigit():
        count = int(collectible_id)
        # Convert "4" -> "strawberry_5"
        final_id = f"strawberry_{count + 1}"
    else:
        # User gave us the ID directly (e.g., "strawberry_5")
        final_id = collectible_id
        
    # Find the collectible
    collectible = next(
        (item for item in CELESTE_COLLECTIBLES
         if item["collectible_id"] == final_id),
        None
    )

    if not collectible:
        return f"Error: Collectible ID '{final_id}' not found. Provide a valid collectible ID from the Forsaken City."

    # Convert to int if needed and validate
    try:
        tier_num = int(hint_tier)
    except (ValueError, TypeError):
        return f"Error: Invalid hint tier '{hint_tier}'. Please use 1, 2, or 3."

    # Validate tier is in valid range
    if tier_num < 1 or tier_num > 3:
        return f"Error: Hint tier must be between 1 and 3. You provided: {tier_num}"

    # Convert to array index (1-based to 0-based)
    tier_index = tier_num - 1

    # Check cache first
    cache_key = collectible_id
    if cache_key not in _hint_cache:
        # Generate tiered hints using LLM
        tiered_hints = generate_tiered_hints(
            collectible["full_instruction"],
            collectible["collectible_type"]
        )
        _hint_cache[cache_key] = tiered_hints
    else:
        tiered_hints = _hint_cache[cache_key]

    # Get the appropriate tier hint
    hint_instruction = tiered_hints[tier_index]

    # Return formatted hint message
    hint_text = (
        f"**Hint (Tier {tier_num}) for {collectible['collectible_type'].title()} "
        f"#{collectible.get('collectible_number', 'N/A')}**\n\n"
        f"{hint_instruction}\n\n"
        f"*Source: {collectible['source']}*"
    )
    
    return hint_text

In [82]:
# Enter the ID of your google sheet here
gsheet_id_default = "1kpbfZkxR288BmRQ7oxaIbGfPrP4xgqDrAcx8OQz6qcg"

Ensure to share with the following account: ``yelpbotvm@mixedinitiative.iam.gserviceaccount.com``

In [83]:
botname = "Celeste GameGenie"
starting_prompt = "Hello! I'm Celeste GameGenie, your AI guide for collecting items and progressing in Celeste. Having trouble reaching the next collectible? Tell me how many strawberries you've collected so far (check the top-left corner in the pause menu if you're not sure), and I'll provide hints and tips to help you find the next ones!"
description = "Celeste GameGenie is an assistant that provides hints and tips for players trying to collect items in the game Celeste. Users can specify which collectible they are aiming for, and what level of hint they would like, and CelesteCollectibleGenie will offer tiered hints to help them progress. The assistant is knowledgeable about various mechanics in the game, such as dashing and jumping, and can provide strategic advice based on the player's current situation. CelesteCollectibleGenie aims to enhance the gaming experience by offering helpful guidance without giving away complete solutions, encouraging players to think critically and improve their skills."

In [84]:
# Helper code to convert the dialogue state to JSON format

from worksheets.utils.annotation import get_agent_action_schemas, get_context_schema
from worksheets.core.dialogue import CurrentDialogueTurn

def convert_to_json(dialogue: list[CurrentDialogueTurn]):
    json_dialogue = []
    for turn in dialogue:
        json_turn = {
            "user": turn.user_utterance,
            "bot": turn.system_response,
            "turn_context": get_context_schema(turn.context),
            "global_context": get_context_schema(turn.global_context),
            "system_action": get_agent_action_schemas(turn.system_action),
            "user_target_sp": turn.user_target_sp,
            "user_target": turn.user_target,
            "user_target_suql": turn.user_target_suql,
        }
        json_dialogue.append(json_turn)
    return json_dialogue

In [85]:
# You need to define the models to use for each componenet of Genie Worksheets.
# You can also load the configurations from a yaml file.

from worksheets.agent.config import Config, AzureModelConfig, OpenAIModelConfig

config = Config(
    semantic_parser = OpenAIModelConfig(
        model_name="gpt-4o-mini",
    ),
    response_generator = OpenAIModelConfig(
        model_name="gpt-4o-mini",
    ),
    knowledge_parser=OpenAIModelConfig(),
    knowledge_base=OpenAIModelConfig(),
    validate_response= False,

    prompt_log_path = "logs.log",
    conversation_log_path= "conv_log.json",
)

In [86]:
# Initialize your llm by providing it the prompts directory and your `.env` file which contains the secrets

from worksheets.llm.prompts import init_llm

# Use paths relative to current directory
prompts_path = os.path.join(os.getcwd(), "genie-worksheets", "src", "worksheets", "prompts")
env_path = os.path.join(os.getcwd(), ".env")

init_llm(prompts_path, env_path)

In [87]:
# Main agent builder that generates the agent for you
from worksheets import AgentBuilder, conversation_loop
import json
from loguru import logger

logger.remove()
logger.add(sys.stderr, level="ERROR")

agent_builder = (
    AgentBuilder(
        name=botname,
        description=description,
        starting_prompt=starting_prompt
    )
    .with_gsheet_specification(gsheet_id_default)
)

agent = agent_builder.build(config)


In [88]:
for ws in agent.runtime.genie_worksheets:
  print(ws)

RequestHint(collectible_id: str, hint_tier: int, confirm_details: bool)


# INTERACTION WITH AGENT

In [46]:
with agent:
  await conversation_loop(agent, debug=True)

Agent: Hello! I am CelesteCollectibleGenie. I can help you with hints and tips for collecting items in Celeste levels. How many strawberries have you collected so far? Check the pause menu in the top-left corner of your game to check.



In [ ]:
import gradio as gr

async def chat_with_genie(message, history):
    """Chat function for Gradio - replicates conversation_loop logic"""
    await agent.generate_next_turn(message)
    
    # Get the last response from dialogue history
    if agent.dlg_history:
        return agent.dlg_history[-1].system_response
    return "Error: No response generated"

# Create the Gradio interface with initial greeting
demo = gr.ChatInterface(
    fn=chat_with_genie,
    title="🍓 Celeste GameGenie",
    description="<h3 style='text-align: center;'><b>Welcome to Celeste GameGenie - Your AI companion for conquering the challenging platforming puzzles in the world of Celeste.</b></h3>",
    chatbot=gr.Chatbot(value=[{"role": "assistant", "content": agent.starting_prompt}])  # Pre-populate with greeting
)

In [ ]:
demo.launch(share=False, inbrowser=True)


* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


request_hint = RequestHint(collectible_id='strawberry_7', hint_tier=1, confirm_details=True)
request_hint.confirm_details = True
